# Common DataFrame manipulations to all storage options 

In [1]:
import os
from openpyxl import load_workbook
import pandas as pd
from datetime import timedelta

BASE_DIR = os.path.dirname(os.path.dirname(os.path.abspath('')))
DATA_DIR = os.path.join(BASE_DIR, 'amerigo_island', 'data')
RAW_DATA_DIR = os.path.join(DATA_DIR, 'raw')
INTERIM_DATA_DIR = os.path.join(DATA_DIR, 'interim')

INPUT_LOAD_AND_VRE_FILENAME = 'load_and_vre.csv'
FINAL_LOAD_AND_VRE_FILENAME = 'final_load_and_vre.csv'

In [2]:
# all input columns as variables for later use
date_colname = 'date'
load_colname = 'load_mw'
solar_gen_colname = 'solar_mw'
wind_gen_colname = 'wind_mw'

load_and_vre_df = pd.read_csv(os.path.join(RAW_DATA_DIR, INPUT_LOAD_AND_VRE_FILENAME))
input_cols = [date_colname, load_colname, wind_gen_colname, solar_gen_colname]
load_and_vre_df = load_and_vre_df.iloc[:, [i for i in range(len(input_cols))]]
load_and_vre_df.columns = input_cols

In [3]:
# all derived columns as variables for later use

### COMMON - columns needed bt legacy gen, batteries, hydrogen
total_vre_gen_colname = 'total_vre_mw'
hourly_vre_gen_wo_storage_colname = 'hourly_vre_gen_wo_storage'
day_of_month_colname = 'day_of_month'
day_of_yr_colname = 'day_of_yr'
week_of_yr_colname = 'wk_of_yr'
month_colname = 'month'
year_colname = 'year'
weekday_colname = 'weekday'
solar_o_and_m_cost_colname = 'solar_o_and_m_cost'
wind_o_and_m_cost_colname = 'wind_o_and_m_cost'
load_vre_diff_colname = 'load_vre_diff'
prev_7_load_colname = 'prev_7_load'
prev_7_load_vre_diff_colname = 'prev_7_load_vre_diff'
prev_7_vre_gen_colname = 'prev_7_vre_gen'
critical_load_mw_colname = 'critical_load_mw'
critical_load_less_vre_colname = 'critical_load_less_vre_mw'
surplus_vre_colname = 'surplus_vre'
electrolyzer_power_input_mw_colname = 'electrolyzer_power_input_mw'
fuel_cell_power_generation_mw_colname = 'fuel_cell_power_generation_mw'
load_after_thermal_base_gen_colname = 'load_after_thermal_base_gen'

### BATTERIES 
cuml_load_since_prev_charge_colname = 'cuml_load_since_prev_charge_mw'
cuml_charge_since_prev_charge_colname = 'cuml_charge_since_prev_discharge_mw'
battery_soc_colname = 'battery_soc'

### HYDROGEN
req_hydrogen_kg_colname = 'reqd_hydrogen_kg'
hydrogen_production_kg_colname = 'hydrogen_prod_kg'
hydrogen_production_m3_colname = 'hydrogen_prod_m3'
hydrogen_h2o_demand_l_colname = 'hydrogen_h2o_demand_l'
hydrogen_h2o_demand_m3_colname = 'hydrogen_h2o_demand_m3'
compression_desal_demand_mw_colname = 'compression_desal_demand_mw' 
h2_demand_comp_desal_kg_colname = 'h2_demand_comp_desal_kg'
cuml_h2_storage_kg_colname = 'cuml_h2_storage_kg'
cuml_hydrogen_demand_kg_colname = 'cuml_hydrogen_demand_kg'
cuml_h2_energy_demand_mwh_colname = 'cuml_h2_energy_demand_mwh'
h2_opex_usd_per_kg_colname = 'h2_opex_usd_per_kg'
h2_opex_electrolyzer_colname = 'h2_opex_electrolyzer'

### BAU - LEGACY GENERATION
thermal_gen_total_mw_colname = 'thermal_gen_total_mw'
thermal_gen_rice_mw_colname = 'thermal_gen_rice_mw'
thermal_gen_combustion_mw_colname = 'thermal_gen_combustion_mw'

In [4]:
# other values/assumptions needed for the analysis
CRITICAL_LOAD_PERC = (1/3)
SOLAR_SCALE_FACTOR = 1
WIND_SCALE_FACTOR = 1

# CAPEX #############################################
SOLAR_INSTALLATION_COST_MM = 50.8
WIND_INSTALLATION_COST_MM = 31.2
H2_FUEL_CELL_COST_MM = 4.23 
H2_ELECTROLYZER_COST_MM = 21.12

# H2 Storage 
H2_STORAGE_CAPEX_USD_PER_KG = 700 # Parks, G., Boyd, R., Cornish, J., & Remick, R. (2014). Hydrogen Station Compression, Storage, and Dispensing Technical Status and Costs: Systems Integration. Related Information: Independent Review 
H2_MAX_STORAGE_CAPACITY_KG = 65000
H2_STORAGE_COST_MM = (H2_STORAGE_CAPEX_USD_PER_KG * H2_MAX_STORAGE_CAPACITY_KG)/ 1_000_000
# H2 Compression
H2_COMPRESSION_COST_MM = 7.32
#####################################################

# O & M #############################################
SOLAR_O_AND_M_COST_KW_PER_YR = 9.1 # kW / yr : https://www.nrel.gov/docs/fy19osti/72399.pdf
WIND_O_AND_M_COST_KW_PER_YR = 44 # kW / yr (ATB - https://atb.nrel.gov/electricity/2019/index.html?t=lw)
FUEL_CELL_O_AND_M_COST_MM = .05 * H2_FUEL_CELL_COST_MM * 1_000_000 # of capital cost (https://www.nrel.gov/docs/fy16osti/65856.pdf)
ELECTROLYZER_O_AND_M_COST_KW_PER_YR = 42 #  (https://www.nrel.gov/docs/fy16osti/65856.pdf)
#####################################################

# Hydrogen storage ##################################
H2_COMPRESSION_CAPEX_KG_H2_PER_DAY = 891 # (HyET Hydrogen. (2020). Electrochemical compression - Technology and performance. Retrieved from http://hyet.nl/hydrogen/technology-and-performance/)

ELEC_EFF_H2_FUEL_CELL_KWH_PER_KG = 19.988333333333
ELEC_EFF_ELECTROLYSIS_KWH_PER_KG = 55.4307635

ENERGY_CONSUMPTION_H2O_DESAL_KWH_PER_M3 = 1.8
DENSITY_H2_300BAR_KG_PER_M3 = 20
DENSITY_H2_30BAR_KG_PER_M3 = 2.38
H2O_ELECTROLYSIS_CONSUMPTION_L_PER_M3 = 1.4

DENSITY_H2_STP_KG_PER_M3 = 0.0813
H2_TAP_WATER_CONSUMPTION_L_PER_M3 = 1.4

ENERGY_CONSUMPTION_H2_KWH_PER_KG = 4
STARTING_H2_IN_STORAGE_KG = 10000
OPEX_H2_USD_PER_KG = 1.08
#####################################################

# Battery storage ##################################
BATTERY_CAPACITY_MWH = 225
BATTERY_POWER_MW = 50
BATTERY_POWER_KW = BATTERY_POWER_MW * 1000
BATTERY_CAPEX = ((BATTERY_CAPACITY_MWH * 1000 * 200) + (BATTERY_POWER_MW * 1000 * 600))
BATT_O_AND_M_COST_PER_KW_YR = .025 * (BATTERY_CAPEX / BATTERY_POWER_KW) #(https://www.nrel.gov/docs/fy19osti/73222.pdf)
THERMAL_BASE_GEN_MW = 32
#####################################################

# Legacy generation options #########################
NOMINAL_HEAT_RATE_BTU_COMBUSTION = 17680
NOMINAL_HEAT_RATE_BTU_RICE = 9827
#####################################################

In [5]:
#$6/kW-yr FOM and a $7.5/kWh-yr



In [6]:
def clean_input_columns(df):

    cleaned_df = df.copy()
    cleaned_df = cleaned_df.dropna(how='all')

    cleaned_df[date_colname] = pd.to_datetime(cleaned_df[date_colname])
    # all solar vals need to be > 0
    cleaned_df[solar_gen_colname] = cleaned_df[solar_gen_colname]\
        .apply(lambda x: x if x > 0 else 0)
    
    cleaned_df[month_colname] = cleaned_df[date_colname].map(lambda x: x.month)
    cleaned_df[day_of_month_colname] = cleaned_df[date_colname].map(lambda x: x.day)
    cleaned_df[day_of_yr_colname] = cleaned_df[date_colname].map(lambda x: x.timetuple().tm_yday)
    
    return cleaned_df

In [7]:
cleaned_load_and_vre_df = clean_input_columns(load_and_vre_df)

In [8]:
cleaned_load_and_vre_df

,date,load_mw,wind_mw,solar_mw,month,day_of_month,day_of_yr
0,2017-01-01 00:00:00,35.1,1.67966,0.0,1,1,1
1,2017-01-01 01:00:00,35.1,1.67966,0.0,1,1,1
2,2017-01-01 02:00:00,34.6,1.23327,0.0,1,1,1
3,2017-01-01 03:00:00,34.2,1.09193,0.0,1,1,1
4,2017-01-01 04:00:00,33.8,1.46510,0.0,1,1,1
...,...,...,...,...,...,...,...
8755,2017-12-31 19:00:00,45.5,6.67250,0.0,12,31,365
8756,2017-12-31 20:00:00,46.5,6.76534,0.0,12,31,365
8757,2017-12-31 21:00:00,45.0,6.20556,0.0,12,31,365
8758,2017-12-31 22:00:00,41.7,8.72619,0.0,12,31,365


In [9]:
assert len(cleaned_load_and_vre_df) == 8760, "Expected one row per hour in year (8760), got {} rows".format(len(load_and_gen_df))

for day_of_yr in cleaned_load_and_vre_df[day_of_yr_colname].unique():
    if len(cleaned_load_and_vre_df[cleaned_load_and_vre_df[day_of_yr_colname] == day_of_yr]) != 24:
        assert ValueError("Day {} has {} values".format(day_of_yr, len(cleaned_load_and_vre_df[cleaned_load_and_vre_df[day_of_yr_colname] == day_of_yr])))
        

In [10]:
def add_common_derived_columns(df):
    
    decorated_df = df.copy()
    
    decorated_df[solar_gen_colname] = decorated_df[solar_gen_colname] * SOLAR_SCALE_FACTOR
    decorated_df[wind_gen_colname] = decorated_df[wind_gen_colname] * WIND_SCALE_FACTOR
    
    decorated_df[solar_o_and_m_cost_colname] = (decorated_df[solar_gen_colname]* 1000) / SOLAR_O_AND_M_COST_KW_PER_YR / 8760
    decorated_df[wind_o_and_m_cost_colname] = (decorated_df[wind_gen_colname]* 1000) / SOLAR_O_AND_M_COST_KW_PER_YR / 8760
    
    decorated_df[total_vre_gen_colname] = \
        decorated_df[wind_gen_colname] + decorated_df[solar_gen_colname]
    
    decorated_df[critical_load_mw_colname] = \
        decorated_df[load_colname] * CRITICAL_LOAD_PERC
    
    decorated_df[critical_load_less_vre_colname] = \
        decorated_df[critical_load_mw_colname] - decorated_df[total_vre_gen_colname]
    
    decorated_df[surplus_vre_colname] = \
        -decorated_df[critical_load_less_vre_colname]
    
    decorated_df[load_vre_diff_colname] = \
        decorated_df[load_colname] \
        - decorated_df[total_vre_gen_colname]
    
    decorated_df[load_vre_diff_colname] = decorated_df[load_vre_diff_colname]\
        .map(lambda x: x if x > 0 else 0)
    
    decorated_df[critical_load_less_vre_colname] = decorated_df[critical_load_less_vre_colname]\
        .map(lambda x: x if x > 0 else 0)

    decorated_df[surplus_vre_colname] = decorated_df[surplus_vre_colname]\
        .map(lambda x: x if x > 0 else 0)
    
    decorated_df[hourly_vre_gen_wo_storage_colname] = decorated_df.apply(
        lambda x: x[total_vre_gen_colname] if x[total_vre_gen_colname] <= x[critical_load_mw_colname] else x[critical_load_mw_colname],
        axis=1
    )
    
    de
    

    return decorated_df

In [11]:
final_load_and_vre_df = add_common_derived_columns(cleaned_load_and_vre_df)
final_load_and_vre_df.to_csv(os.path.join(INTERIM_DATA_DIR, FINAL_LOAD_AND_VRE_FILENAME), index=False)

In [13]:
final_load_and_vre_df.describe()

,load_mw,wind_mw,solar_mw,month,day_of_month,day_of_yr,solar_o_and_m_cost,wind_o_and_m_cost,total_vre_mw,critical_load_mw,critical_load_less_vre_mw,surplus_vre,load_vre_diff,hourly_vre_gen_wo_storage
count,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000
mean,41.792226,5.711493,5.406281,6.526027,15.720548,183.000000,0.067819,0.071648,11.117774,13.930742,5.242575,2.429608,30.674452,8.688167
std,5.316242,4.145345,7.172350,3.448048,8.796749,105.372043,0.089974,0.052001,8.892676,1.772081,4.960255,4.512237,8.485789,5.400522
min,31.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,10.333333,0.000000,0.000000,1.476600,0.000000
25%,37.000000,2.346448,0.000000,4.000000,8.000000,92.000000,0.000000,0.029435,3.499253,12.333333,0.000000,0.000000,24.841575,3.499253
50%,42.800000,4.740495,0.000000,7.000000,16.000000,183.000000,0.000000,0.059467,8.709935,14.266667,4.770503,0.000000,30.984805,8.709935
75%,46.200000,8.400110,11.221750,10.000000,23.000000,274.000000,0.140772,0.105375,17.417063,15.400000,9.686101,3.225787,36.757332,14.200000
max,55.800000,15.098400,22.877000,12.000000,31.000000,365.000000,0.286981,0.189402,37.832400,18.600000,17.061347,24.123400,52.629400,17.600000


# Hydrogen-specific analysis

In [12]:
def calc_h2_cols(df):
    
    decorated_df = df.copy()
    
    decorated_df[req_hydrogen_kg_colname] = \
        (decorated_df[critical_load_less_vre_colname] * 1000)/ ELEC_EFF_H2_FUEL_CELL_KWH_PER_KG
    
    decorated_df[hydrogen_production_kg_colname] = \
        (decorated_df[surplus_vre_colname] * 1000) / ELEC_EFF_ELECTROLYSIS_KWH_PER_KG
    
    decorated_df[h2_opex_usd_per_kg_colname] = \
        decorated_df[hydrogen_production_kg_colname] * OPEX_H2_USD_PER_KG
    
    decorated_df[hydrogen_production_m3_colname] = \
        decorated_df[hydrogen_production_kg_colname] / DENSITY_H2_30BAR_KG_PER_M3
    
    decorated_df[hydrogen_h2o_demand_l_colname] = \
        decorated_df[hydrogen_production_m3_colname] * H2O_ELECTROLYSIS_CONSUMPTION_L_PER_M3
    
    decorated_df[hydrogen_h2o_demand_m3_colname] = \
        (decorated_df[hydrogen_production_kg_colname] / DENSITY_H2_STP_KG_PER_M3) \
        * (H2_TAP_WATER_CONSUMPTION_L_PER_M3 / 1000)
    
    decorated_df[compression_desal_demand_mw_colname] = \
        (decorated_df[hydrogen_production_kg_colname] * ENERGY_CONSUMPTION_H2_KWH_PER_KG / 1000) \
        + (decorated_df[hydrogen_h2o_demand_m3_colname] * ENERGY_CONSUMPTION_H2O_DESAL_KWH_PER_M3 / 1000)
    
    decorated_df[h2_demand_comp_desal_kg_colname] = \
        (decorated_df[compression_desal_demand_mw_colname] * 1000)/ ELEC_EFF_H2_FUEL_CELL_KWH_PER_KG
    
    decorated_df.sort_values(date_colname)
    
# calc cuml h2 demand
    decorated_df[cuml_hydrogen_demand_kg_colname] = None
    decorated_df[cuml_h2_storage_kg_colname] = None
    decorated_df.reset_index()
    for idx, row in decorated_df.iterrows():
        
        if idx == 0:
            h2_demand_val = row[req_hydrogen_kg_colname]
            h2_storage_val = STARTING_H2_IN_STORAGE_KG \
                - row[req_hydrogen_kg_colname] \
                - row[h2_demand_comp_desal_kg_colname] \
                + row[hydrogen_production_kg_colname]

        else : #
            
            h2_demand_val = \
                decorated_df.loc[idx - 1, cuml_hydrogen_demand_kg_colname] \
                + row[req_hydrogen_kg_colname]\
                - row[hydrogen_production_kg_colname]
            
            h2_storage_val = decorated_df.loc[idx - 1, cuml_h2_storage_kg_colname] \
                - row[req_hydrogen_kg_colname] \
                - row[h2_demand_comp_desal_kg_colname] \
                + row[hydrogen_production_kg_colname]
        
        decorated_df.at[idx, cuml_hydrogen_demand_kg_colname] = h2_demand_val
        decorated_df.at[idx, cuml_h2_storage_kg_colname] = h2_storage_val
    
        
    decorated_df[electrolyzer_power_input_mw_colname] = \
        decorated_df[hydrogen_production_kg_colname] * ELEC_EFF_ELECTROLYSIS_KWH_PER_KG/ 1000
    decorated_df[cuml_h2_energy_demand_mwh_colname] = \
        (decorated_df[cuml_hydrogen_demand_kg_colname] * ELEC_EFF_H2_FUEL_CELL_KWH_PER_KG) / 1000
    decorated_df[fuel_cell_power_generation_mw_colname] = \
        (decorated_df[req_hydrogen_kg_colname] \
        + decorated_df[h2_demand_comp_desal_kg_colname]) \
        * (ELEC_EFF_H2_FUEL_CELL_KWH_PER_KG / 1000)
    
    decorated_df[h2_opex_electrolyzer_colname] = \
        decorated_df[electrolyzer_power_input_mw_colname] * 1000 \
        * ELECTROLYZER_O_AND_M_COST_KW_PER_YR / 8760

    return decorated_df

In [13]:
h2_hourly_df = calc_h2_cols(final_load_and_vre_df)

h2_daily_df = h2_hourly_df.groupby(day_of_yr_colname).sum().reset_index()
h2_daily_df = h2_daily_df.drop([month_colname, day_of_month_colname], axis=1)

# Add common columns to daily DataFrame

In [14]:
def sum_prev_rows(row, df, lookback_colname, agg_colname, days = 7):

    start_idx = row[lookback_colname] - days    
    return_val = df.loc[start_idx:row[lookback_colname] - 1, agg_colname].sum() if start_idx >= 0 else None
    
    return return_val

def add_daily_common_cols(df):
    
    decorated_df = df.copy()

    decorated_df[prev_7_load_colname] = decorated_df\
        .apply(sum_prev_rows, axis=1, args=(decorated_df, day_of_yr_colname, load_colname,))

    decorated_df[prev_7_load_vre_diff_colname] = decorated_df\
        .apply(sum_prev_rows, axis=1, args=(decorated_df, day_of_yr_colname, load_vre_diff_colname,))

    decorated_df[prev_7_vre_gen_colname] = decorated_df\
        .apply(sum_prev_rows, axis=1, args=(decorated_df, day_of_yr_colname, total_vre_gen_colname,))

    return decorated_df

In [15]:
daily_df = add_daily_common_cols(h2_daily_df)

# Battery-specific analysis

In [16]:
battery_soc_colname = 'battery_soc_mwh'
battery_curtailment_colname = 'battery_curtailment'
battery_o_and_m_cost_colname = 'battery_o_and_m'
load_after_thermal_base_gen_mw_colname = 'load_after_thermal_base_gen_mw'
remaining_load_after_vre_prod_mw_colname = 'remaining_load_after_vre_prod_mw'

battery_capacity_mwh_config_prop = "battery_capacity_mwh"
battery_power_mw_config_prop = "battery_power_mw"

def add_battery_cols(hourly_df, battery_config):
    
    decorated_df = hourly_df.copy()
    
    battery_capacity = battery_config[battery_capacity_mwh_config_prop]

    decorated_df[load_after_thermal_base_gen_mw_colname] = \
        decorated_df[load_colname] - THERMAL_BASE_GEN_MW
        
    decorated_df[remaining_load_after_vre_prod_mw_colname] = \
        decorated_df[load_after_thermal_base_gen_mw_colname] \
        - decorated_df[total_vre_gen_colname]
    
    decorated_df[remaining_load_after_vre_prod_mw_colname] = \
        decorated_df[remaining_load_after_vre_prod_mw_colname].map(lambda x: x if x > 0 else 0)
    
    decorated_df[battery_soc_colname] = None
    decorated_df[battery_curtailment_colname] = None

    decorated_df.reset_index()
    for idx, row in decorated_df.iterrows():

        if idx == 0:
            
            soc_val = \
                battery_capacity \
                - row[remaining_load_after_vre_prod_mw_colname] \
                + row[surplus_vre_colname]
            batt_curtailment_val = 0
                
        else:
            soc_val = \
                decorated_df.loc[idx - 1, battery_soc_colname]\
                - row[remaining_load_after_vre_prod_mw_colname]\
                + row[surplus_vre_colname]
            soc_val = soc_val if soc_val < battery_capacity else battery_capacity
            soc_val = soc_val if soc_val > 0 else 0
            
            batt_curtailment_val = \
                decorated_df.loc[idx - 1, battery_soc_colname] \
                - row[remaining_load_after_vre_prod_mw_colname]\
                + row[surplus_vre_colname]\
                - soc_val
            
            batt_curtailment_val = batt_curtailment_val if batt_curtailment_val > 0 else 0

        
        decorated_df.at[idx, battery_soc_colname] = soc_val
        decorated_df.at[idx, battery_curtailment_colname] = batt_curtailment_val

    decorated_df[battery_o_and_m_cost_colname] = \
        decorated_df[remaining_load_after_vre_prod_mw_colname] * 1000 \
        * BATT_O_AND_M_COST_PER_KW_YR \
        / 8760
    
    decorated_df[battery_soc_colname] = decorated_df[battery_soc_colname].astype(float)
    decorated_df[battery_curtailment_colname] = decorated_df[battery_curtailment_colname].astype(float)
    return decorated_df

In [17]:
battery_config = {
    battery_capacity_mwh_config_prop : BATTERY_CAPACITY_MWH,
    battery_power_mw_config_prop : BATTERY_POWER_MW
}

In [18]:
battery_hourly_df = add_battery_cols(h2_hourly_df, battery_config)

In [19]:
battery_hourly_df

,date,load_mw,wind_mw,solar_mw,month,day_of_month,day_of_yr,solar_o_and_m_cost,wind_o_and_m_cost,total_vre_mw,...,cuml_h2_storage_kg,electrolyzer_power_input_mw,cuml_h2_energy_demand_mwh,fuel_cell_power_generation_mw,h2_opex_electrolyzer,load_after_thermal_base_gen_mw,remaining_load_after_vre_prod_mw,battery_soc_mwh,battery_curtailment,battery_o_and_m
0,2017-01-01 00:00:00,35.1,1.67966,0.0,1,1,1,0.0,0.021071,1.67966,...,9498.69,0.0,10.0203,10.020340,0.0,3.1,1.42034,223.57966,0.0,6.080223
1,2017-01-01 01:00:00,35.1,1.67966,0.0,1,1,1,0.0,0.021071,1.67966,...,8997.38,0.0,20.0407,10.020340,0.0,3.1,1.42034,222.15932,0.0,6.080223
2,2017-01-01 02:00:00,34.6,1.23327,0.0,1,1,1,0.0,0.015471,1.23327,...,8482.08,0.0,30.3407,10.300063,0.0,2.6,1.36673,220.79259,0.0,5.850728
3,2017-01-01 03:00:00,34.2,1.09193,0.0,1,1,1,0.0,0.013698,1.09193,...,7966.37,0.0,40.6488,10.308070,0.0,2.2,1.10807,219.68452,0.0,4.743450
4,2017-01-01 04:00:00,33.8,1.46510,0.0,1,1,1,0.0,0.018379,1.46510,...,7476.01,0.0,50.4504,9.801567,0.0,1.8,0.33490,219.34962,0.0,1.433647
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,2017-12-31 19:00:00,45.5,6.67250,0.0,12,31,365,0.0,0.083703,6.67250,...,-1.97962e+06,0.0,38221.5,8.494167,0.0,13.5,6.82750,158.77412,0.0,29.227312
8756,2017-12-31 20:00:00,46.5,6.76534,0.0,12,31,365,0.0,0.084868,6.76534,...,-1.98006e+06,0.0,38230.2,8.734660,0.0,14.5,7.73466,151.03946,0.0,33.110702
8757,2017-12-31 21:00:00,45.0,6.20556,0.0,12,31,365,0.0,0.077846,6.20556,...,-1.9805e+06,0.0,38239,8.794440,0.0,13.0,6.79444,144.24502,0.0,29.085788
8758,2017-12-31 22:00:00,41.7,8.72619,0.0,12,31,365,0.0,0.109466,8.72619,...,-1.98076e+06,0.0,38244.2,5.173810,0.0,9.7,0.97381,143.27121,0.0,4.168707


# BAU Analysis

In [20]:
thermal_gen_total_mw_colname = 'thermal_gen_total_mw'
thermal_gen_total_kwh_colname = 'thermal_gen_total_kwh'

'''
values for RICE taken from: https://www.eergy.gov/sites/prod/files/2016/09/f33/CHP-Recip%20Engines.pdf
assumed to be System #5 in paper above, for two reasons:
- System #5 has highest capacity
- System #5 has O&M similar to what is specified in assignment
'''

therm_rice_config = {
    "o_and_m" : .0105, # in cents / kwh
    "nominal_heat_rate_btu" : NOMINAL_HEAT_RATE_BTU_RICE, # in BTU / kWh
    "fuel_cost" : 2.50, # in $ / gallon
    "thermal_efficiency" : .35,
    "electric_efficiency" : .419,
    "total_efficiency" : .769,
    "nominal_heat_rate_gal" : NOMINAL_HEAT_RATE_BTU_RICE / 139_000,
    "column_prepend" : "rice"
}

'''
values for combustion engine taken from: https://www.epa.gov/sites/production/files/2015-07/documents/catalog_of_chp_technologies_section_3._technology_characterization_-_combustion_turbines.pdf
Specifically, Table 3-2. 
Numbers reflect System #1 in paper above, for two reasons:
- System #1 has closest electric heat rate to what is specified in assignment PDF

Annual Tech Baseline - EIA
'''


therm_combustion_config = {
    "o_and_m" : .0046, # in $ / kWh
    "nominal_heat_rate_btu" : NOMINAL_HEAT_RATE_BTU_COMBUSTION, # in BTU / kWh 
    "fuel_cost" : 2.50, # in $ / gallon
    "thermal_efficiency" : .42,
    "electric_efficiency" : .24,
    "total_efficiency" : .657,
    "nominal_heat_rate_gal" : NOMINAL_HEAT_RATE_BTU_COMBUSTION / 139_000,
    "column_prepend" : "combustion"
}

In [21]:
therm_combustion_config

{'o_and_m': 0.0046,
 'nominal_heat_rate_btu': 17680,
 'fuel_cost': 2.5,
 'thermal_efficiency': 0.42,
 'electric_efficiency': 0.24,
 'total_efficiency': 0.657,
 'nominal_heat_rate_gal': 0.12719424460431655,
 'column_prepend': 'combustion'}

In [22]:
therm_rice_config

{'o_and_m': 0.0105,
 'nominal_heat_rate_btu': 9827,
 'fuel_cost': 2.5,
 'thermal_efficiency': 0.35,
 'electric_efficiency': 0.419,
 'total_efficiency': 0.769,
 'nominal_heat_rate_gal': 0.07069784172661871,
 'column_prepend': 'rice'}

In [23]:
def calc_bau_cols(hourly_df, config):
    
    decorated_df = hourly_df.copy()
    efficiency_key = 'total_efficiency'
    
    col_prepend = config['column_prepend']
    o_and_m = config['o_and_m']
    nominal_heat_rate_gal = config['nominal_heat_rate_gal']
    fuel_cost = config['fuel_cost']
    efficiency = config[efficiency_key]
    
    decorated_df[thermal_gen_total_mw_colname] = decorated_df[load_colname] - decorated_df[total_vre_gen_colname]
    decorated_df[thermal_gen_total_kwh_colname] = decorated_df[thermal_gen_total_mw_colname] * 1000
    
    decorated_df['{}_fuel_cost'.format(col_prepend)] = \
        decorated_df[thermal_gen_total_kwh_colname] \
        * nominal_heat_rate_gal \
        * fuel_cost
    
    decorated_df['{}_o_and_m_cost'.format(col_prepend)] = \
        decorated_df[thermal_gen_total_kwh_colname] * o_and_m
    
    decorated_df['{}_total_cost'.format(col_prepend)] = decorated_df['{}_o_and_m_cost'.format(col_prepend)] + decorated_df['{}_fuel_cost'.format(col_prepend)]
    
    return decorated_df
    

In [24]:
rice_df = calc_bau_cols(battery_hourly_df, therm_rice_config)
full_df = calc_bau_cols(rice_df, therm_combustion_config)

In [34]:
full_df[["rice_fuel_cost", "combustion_fuel_cost"]].sum()

rice_fuel_cost          4.749272e+07
combustion_fuel_cost    8.544534e+07
dtype: float64

In [25]:
full_df.to_csv('full_df.csv', index=False)

In [26]:
opex_cols = [
    h2_opex_electrolyzer_colname, 
    h2_opex_usd_per_kg_colname, 
    "rice_o_and_m_cost", 
    "combustion_o_and_m_cost"       
]

for opex_col in opex_cols:
    print("{} : ${:0,.0f}".format(opex_col, full_df[opex_col].sum()))
    
print("\n--- fixed O&M costs")
print("solar_o_and_m: ${:0,.0f}".format(SOLAR_O_AND_M_COST_KW_PER_YR * 11125))
print("wind_o_and_m: ${:0,.0f}".format(WIND_O_AND_M_COST_KW_PER_YR * 5711))
print("fuel_cell_o_and_m : ${:0,.0f}".format(FUEL_CELL_O_AND_M_COST_MM))
print("battery_o_and_m: ${:0,.0f}".format(BATTERY_CAPEX * .025))


h2_opex_electrolyzer : $102,044
h2_opex_usd_per_kg : $414,680
rice_o_and_m_cost : $2,821,436
combustion_o_and_m_cost : $1,236,058

--- fixed O&M costs
solar_o_and_m: $101,238
wind_o_and_m: $251,284
fuel_cell_o_and_m : $211,500
battery_o_and_m: $1,875,000


# Scratch
##### Unsure if needed. Keeping just in case

In [27]:
resilience_case_load_daynum = \
    daily_df[daily_df[prev_7_load_colname] == daily_df[prev_7_load_colname].max()].to_dict('r')[0][day_of_yr_colname]

resilience_case_vre_load_diff_daynum = \
    daily_df[daily_df[prev_7_load_vre_diff_colname] == daily_df[prev_7_load_vre_diff_colname].max()].to_dict('r')[0][day_of_yr_colname]

resilience_case_max_demand_daynum = \
    final_load_and_vre_df[final_load_and_vre_df[load_colname] == final_load_and_vre_df[load_colname].max()].to_dict('r')[0][day_of_yr_colname]


In [28]:
def add_storage_input_cols(df, daynum):
    
    decorated_df = df.copy()
    
    decorated_df = decorated_df[
        (decorated_df[day_of_yr_colname] <= daynum)
        & (decorated_df[day_of_yr_colname] >= daynum - 6)
    ]
    
    decorated_df = decorated_df.reset_index(drop=True)
    
    decorated_df.sort_values(date_colname)
    decorated_df[cuml_load_since_prev_charge_colname] = None
    
    for idx, row in decorated_df.iterrows():
        
        if idx == 0:
            val = row[critical_load_less_vre_colname]
        elif row[critical_load_less_vre_colname] > 0:
            val = decorated_df.loc[idx - 1, cuml_load_since_prev_charge_colname] + row[critical_load_less_vre_colname]
        else:
            val = 0
            
        decorated_df.at[idx, cuml_load_since_prev_charge_colname] = val
        
    return decorated_df

In [29]:
resil_max_week_load_df = add_storage_input_cols(h2_hourly_df, resilience_case_load_daynum)
resil_vre_load_diff_df = add_storage_input_cols(h2_hourly_df, resilience_case_vre_load_diff_daynum)
resil_max_demand_df = add_storage_input_cols(h2_hourly_df, resilience_case_max_demand_daynum)

In [30]:
resil_max_week_load_df

,date,load_mw,wind_mw,solar_mw,month,day_of_month,day_of_yr,solar_o_and_m_cost,wind_o_and_m_cost,total_vre_mw,...,hydrogen_h2o_demand_m3,compression_desal_demand_mw,h2_demand_comp_desal_kg,cuml_hydrogen_demand_kg,cuml_h2_storage_kg,electrolyzer_power_input_mw,cuml_h2_energy_demand_mwh,fuel_cell_power_generation_mw,h2_opex_electrolyzer,cuml_load_since_prev_charge_mw
0,2017-08-14 00:00:00,40.8,3.391490,0.000,8,14,226,0.000000,0.042545,3.391490,...,0.0,0.0,0.0,1.02197e+06,-1.06603e+06,0.0,20427.5,10.208510,0.0,10.2085
1,2017-08-14 01:00:00,39.2,4.369580,0.000,8,14,226,0.000000,0.054814,4.369580,...,0.0,0.0,0.0,1.02241e+06,-1.06646e+06,0.0,20436.2,8.697087,0.0,18.9056
2,2017-08-14 02:00:00,38.3,4.737650,0.000,8,14,226,0.000000,0.059432,4.737650,...,0.0,0.0,0.0,1.02281e+06,-1.06686e+06,0.0,20444.3,8.029017,0.0,26.9346
3,2017-08-14 03:00:00,37.5,5.029210,0.000,8,14,226,0.000000,0.063089,5.029210,...,0.0,0.0,0.0,1.02318e+06,-1.06724e+06,0.0,20451.7,7.470790,0.0,34.4054
4,2017-08-14 04:00:00,37.5,4.012080,0.000,8,14,226,0.000000,0.050330,4.012080,...,0.0,0.0,0.0,1.02361e+06,-1.06766e+06,0.0,20460.2,8.487920,0.0,42.8933
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,2017-08-20 19:00:00,50.3,0.886276,0.054,8,20,232,0.000677,0.011118,0.940276,...,0.0,0.0,0.0,1.05694e+06,-1.10231e+06,0.0,21126.4,15.826391,0.0,253.142
164,2017-08-20 20:00:00,50.2,0.699870,0.000,8,20,232,0.000000,0.008780,0.699870,...,0.0,0.0,0.0,1.05774e+06,-1.10311e+06,0.0,21142.4,16.033463,0.0,269.175
165,2017-08-20 21:00:00,48.5,1.539350,0.000,8,20,232,0.000000,0.019310,1.539350,...,0.0,0.0,0.0,1.05847e+06,-1.10384e+06,0.0,21157.1,14.627317,0.0,283.802
166,2017-08-20 22:00:00,46.6,0.829267,0.000,8,20,232,0.000000,0.010403,0.829267,...,0.0,0.0,0.0,1.05921e+06,-1.10458e+06,0.0,21171.8,14.704066,0.0,298.506


In [31]:
def calc_soc_cols(df, config):
    
    decorated_df = df.copy()
    batt_energy_mwh = config[batt_energy_mwh_keyname]
    
    decorated_df[battery_soc_colname] = None
    decorated_df.reset_index(drop=True)
    
    for idx, row in decorated_df.iterrows():
        
        if idx == 0:
            val = batt_energy_mwh - row[critical_load_less_vre_colname] + row[charge_surplus_colname]
        else:
            charge = decorated_df.loc[idx - 1, battery_soc_colname] - row[critical_load_less_vre_colname] + row[charge_surplus_colname] 
            val = charge if charge < batt_energy_mwh else batt_energy_mwh
            
        decorated_df.at[idx, battery_soc_colname] = val
    
    return decorated_df